# Efficient Fine-Tuning of gemma 27b with Unsloth
nce.

### Key Features

- **Model Size**: 14.7 billion parameters, with 13.1 billion non-embedding parameters.
- **Architecture**: Utilizes transformers with multiple layers (48 layers) and attention heads (40 for queries and 8 for keys/values).
- **Context Length**: Supports long contexts of up to 131,072 tokens, allowing it to handle extensive code and text inputs effectively.
- **Training Tokens**: Trained on a vast dataset of 5.5 trillion tokens, which includes a variety of source code and synthetic data.ons.


In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
from huggingface_hub import login
from getpass import getpass

hf_token = getpass("Enter your Hugging Face token: ")
login(hf_token)

"""Login to Weights & Biases using the API key to initiate the project.

"""

import wandb
from getpass import getpass

wb_token = getpass("Enter your W&B API token: ")
wandb.login(key=wb_token)
run = wandb.init(
    project='Fine-tuning full deepseak-R1-on-Solidity-detection',
    job_type="training",
    anonymous="allow"
)


Enter your Hugging Face token:  ········
Enter your W&B API token:  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: quynhnhu170218 (quynhnhu170218-uit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None          # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True   # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-Distill-Llama-8B", #thay model mong muon
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-05-09 16:16:50.843046: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746807411.069183      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746807411.134425      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

# post-inference

# Data prep

In [4]:
prompt_style = """
Below is an instruction describing a task, followed by a smart contract code snippet. Analyze the code and provide a structured response identifying security vulnerabilities and corresponding remediations.



### Instruction:
You are a smart contract security assistant with deep expertise in Ethereum, Solidity, and DeFi security. Analyze the provided code snippet for vulnerabilities. For each identified issue, give a detailed description, a severity level, and a concrete recommendation to fix or mitigate it.


### Question:
List all the vulnerabilities in this smart contract, and provide recommendations to remediate the issues.

### Code:
{}

### Response:
### Analysis
- Vulnerability description: {}
  - Recommendation: {}
  - type: {}
"""

In [5]:
from datasets import load_dataset

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    codes = examples["code"]
    descriptions = examples["description"]
    recommendations = examples["recommendation"]
    types = examples["type"]
    texts = []
    for code, des, rec, tov in zip(codes, descriptions, recommendations, types):
        text = prompt_style.format(code, des, rec, tov) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}


dataset = load_dataset("msc-smart-contract-auditing/audits-with-reasons", split="train")
# dataset = dataset.select(range(5000, 10000))
dataset = dataset.map(formatting_prompts_func, batched=True)

README.md:   0%|          | 0.00/3.35k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.23M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/743k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2472 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/437 [00:00<?, ? examples/s]

Map:   0%|          | 0/2472 [00:00<?, ? examples/s]

In [6]:
try:
    print("### Example Prompt:")
    print(dataset["text"][0])
except IndexError:
    print("Dataset is empty or formatting failed.")

### Example Prompt:

Below is an instruction describing a task, followed by a smart contract code snippet. Analyze the code and provide a structured response identifying security vulnerabilities and corresponding remediations.



### Instruction:
You are a smart contract security assistant with deep expertise in Ethereum, Solidity, and DeFi security. Analyze the provided code snippet for vulnerabilities. For each identified issue, give a detailed description, a severity level, and a concrete recommendation to fix or mitigate it.


### Question:
List all the vulnerabilities in this smart contract, and provide recommendations to remediate the issues.

### Code:
```\nrequire(block.timestamp >= _d.withdrawalRequestTime + TBTCConstants.getIncreaseFeeTimer(), "Fee increase not yet permitted");\n```\n```\nuint256 _previousOutputValue = DepositUtils.bytes8LEToUint(_previousOutputValueBytes);\n_newOutputValue = DepositUtils.bytes8LEToUint(_newOutputValueBytes);\nrequire(_previousOutputValue.sub

## Model Training

In [7]:
train_prompt_style = """
Below is an instruction describing a task, followed by a smart contract code snippet. Analyze the code and provide a structured response identifying security vulnerabilities and corresponding remediations.



### Instruction:
You are a smart contract security assistant with deep expertise in Ethereum, Solidity, and DeFi security. Analyze the provided code snippet for vulnerabilities. For each identified issue, give a detailed description, a severity level, and a concrete recommendation to fix or mitigate it.


### Question:
List all the vulnerabilities in this smart contract, and provide recommendations to remediate the issues.

### Code:
{}

### Response:
### Analysis
- Vulnerability description: {}
  - Recommendation: {}
  - type: {}
"""

In [8]:
# === Enable LoRA ===
model = FastLanguageModel.get_peft_model(
    model,
    r=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=42,
    use_rslora=False,
)

Unsloth 2025.4.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        # max_steps=60,
        num_train_epochs = 1,  # Huấn luyện qua 1 epoch qua toàn bộ k cần tính max_step
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=42,
        output_dir="outputs",
        logging_strategy="steps",      # log every few steps
        logging_steps=1,               # how often to log
        save_strategy="no",            # avoid saving checkpoints if not needed
        disable_tqdm=False,            # enable progress bar
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2472 [00:00<?, ? examples/s]

In [10]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,472 | Num Epochs = 1 | Total steps = 154
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 83,886,080/8,000,000,000 (1.05% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.575600
2,3.704600
3,2.074600
4,1.190100


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.84 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.55 GiB is free. Process 2578 has 13.18 GiB memory in use. Of the allocated memory 12.85 GiB is allocated by PyTorch, and 174.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
train_prompt_style = """
Below is an instruction describing a task, followed by a smart contract code snippet. Analyze the code and provide a structured response identifying security vulnerabilities and corresponding remediations.



### Instruction:
You are a smart contract security assistant with deep expertise in Ethereum, Solidity, and DeFi security. Analyze the provided code snippet for vulnerabilities. For each identified issue, give a detailed description, a severity level, and a concrete recommendation to fix or mitigate it.


### Question:
List all the vulnerabilities in this smart contract, and provide recommendations to remediate the issues.

### Code:
{}

### Response:
 {}
"""

In [ ]:
code = """
what is the vulnerabilities of this code snippet:
\nrequire(block.timestamp >= _d.withdrawalRequestTime + TBTCConstants.getIncreaseFeeTimer(), "Fee increase not yet permitted");\n```\n```\nuint256 _previousOutputValue = DepositUtils.bytes8LEToUint(_previousOutputValueBytes);\n_newOutputValue = DepositUtils.bytes8LEToUint(_newOutputValueBytes);\nrequire(_previousOutputValue.sub(_newOutputValue) == _d.initialRedemptionFee, "Not an allowed fee step");\n```\n```\nrequire(\n    _d.utxoSize().sub(_fundingOutputValue) <= _d.initialRedemptionFee * 5,\n    "Fee unexpectedly very high"\n);\
"""

In [ ]:
FastLanguageModel.for_inference(model)
inputs = tokenizer([train_prompt_style.format(code, "")], return_tensors="pt").to("cuda")
outputs = model.generate(input_ids=inputs.input_ids, attention_mask=inputs.attention_mask, max_new_tokens=1024)
print("\n### Post-finetuning Inference Example:")
print(tokenizer.batch_decode(outputs)[0].split("### Response:")[1])

In [ ]:
model.save_pretrained("DeepSeek-R1-Distill-Llama-8B") # Local saving
tokenizer.save_pretrained("DeepSeek-R1-Distill-Llama-8B")

# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

In [ ]:
new_model_online = "Nhudang/DeepSeek-R1-Distill-Llama-8B"
model.push_to_hub(new_model_online) # Online saving
tokenizer.push_to_hub(new_model_online) # Online saving